Este colab fue desarrollado por Arnold Charry Armero.

# Problema de Asignación

Se desea asignar a cada persona a máximo un proyecto y cada proyecto debe ser ejecutado una vez. Realice un modelo de programación entera que realice esto.

## Conjuntos

$ i = \textrm{Conjunto de personas, indexado por} \:  i$

$ j = \textrm{Conjunto de proyectos, indexado por} \:  j$

## Parámetros

$ c_{ij} = \textrm{Costo de asignar a la persona } i \textrm{ al proyecto } j$

## Variables de Decisión

$ x_{ij} = \textrm{1 si la persona } i \textrm{ es asignada al proyecto }j \textrm{, 0 lo contrario }$

## Función Objetivo

$$ \text{min} \: \sum_{i = 1}^{n}\sum_{j = 1}^{m}c_{ij} \: x_{ij}$$

## Restricciones

Una persona puede como máximo hacer un proyecto,

$$ \sum_{j = 1}^{n}x_{ij} ≤ 1, \: \forall i $$

Cada proyecto se debe realizar una vez,

$$ \sum_{i = 1}^{n}x_{ij} = 1, \: \forall j $$

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 69.3 MB/s eta 0:00:00


In [ ]:
# Se importan las librerías
import pandas as pd
from gurobipy import *

In [ ]:
# Conjuntos
personas = range(4)
proyectos = range(3)

In [ ]:
# Parámetros
C = [[11, 14, 6],
     [8, 10, 11],
     [9, 12, 7],
     [10, 13, 8]]

In [ ]:
# Crear la instancia del modelo
model = Model("Asignación_Personas")

# Inicialización de las variables de decisión
X = model.addVars(personas, proyectos, vtype=GRB.BINARY, name="X")

# Agregar la función objetivo
model.setObjective(quicksum(C[i][j] * X[i, j] for i in personas for j in proyectos), GRB.MINIMIZE)

# Agregar las restricciones

for i in personas:
    model.addConstr(quicksum(X[i, j] for j in proyectos) <= 1, name=f"Persona_{i}")

for j in proyectos:
  model.addConstr(quicksum(X[i, j] for i in personas) == 1, name=f"Proyecto_{j}")

# Resolver el problema
model.optimize()

# Imprimir el estado del problema
if model.Status == GRB.OPTIMAL:
    print("Status: Optimal")

# Resultados
for var in model.getVars():
  print(var.VarName, "=", var.X)

print(f"\nFunción Objetivo = {model.objVal:.2f}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x45d40bec
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 34.0000000
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 2.500000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      25.0000000   25.00000  0.00%

In [ ]:
# Crear un DataFrame vacío con ceros
asignacion = pd.DataFrame(0, index=personas, columns=proyectos)

# Llenar la tabla con los valores de X
for i in personas:
    for j in proyectos:
        asignacion.loc[i, j] = int(X[i, j].X)

# Mostrar el DataFrame
print("\nTabla de asignación (1 = persona asignada al proyecto):")
display(asignacion)


Tabla de asignación (1 = persona asignada al proyecto):


,0,1,2
0,0,0,1
1,0,1,0
2,1,0,0
3,0,0,0
